In [1]:
#Load libraries
import os
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI, ChatAnthropic
from langchain.llms import AzureOpenAI, LlamaCpp
from langchain.embeddings import OpenAIEmbeddings
from langchain import PromptTemplate, LLMChain

from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

In [2]:
llm_gpt_35_turbo = AzureChatOpenAI(

        deployment_name="gpt-35-turbo",

        model_name="gpt-35-turbo",

        temperature= 0.7,

        openai_api_version="2023-05-15",

        openai_api_key= "cd826423871544a486d616f14805725a",

        openai_api_base="https://testavinx.openai.azure.com/",

        openai_api_type="azure",

    )

In [3]:
embeddings = OpenAIEmbeddings(deployment="embedding1",

        model="text-embedding-ada-002",

        openai_api_version="2023-05-15",

        openai_api_key= "cd826423871544a486d616f14805725a",

        openai_api_base="https://testavinx.openai.azure.com/",

        openai_api_type="azure",

        chunk_size = 1)

In [4]:
template = """Defn: An entity is an age (age), gender of the person (sex), history (history), symptomns of the disease (symptoms), the duration of the 
                        symptoms (duration symptom), disease that has been diagnosed (diagnosis), treament of the disease (treatment)
                        "Abstract concepts, processes and adjectives are not entities"
            Example 1: CASE: A 28-year-old previously healthy man presented with a 6-week history of palpitations.
                        The symptoms occurred during rest, 2–3 times per week, lasted up to 30 minutes at a time and were associated with dyspnea.
                        Except for a grade 2/6 holosystolic tricuspid regurgitation murmur (best heard at the left sternal border with inspiratory
                        accentuation), physical examination yielded unremarkable findings. An electrocardiogram (ECG) revealed normal sinus rhythm
                        and a Wolff– Parkinson– White pre-excitation pattern, produced by a right-sided accessory pathway.
                        Transthoracic echocardiography demonstrated the presence of Ebstein's anomaly of the tricuspid valve, with apical displacement 
                        of the valve and formation of an “atrialized” right ventricle (a functional unit between the right atrium and the inlet [inflow] portion of the right ventricle) (Fig.2).
                        The anterior tricuspid valve leaflet was elongated (Fig.2C, arrow), whereas the septal leaflet was rudimentary (Fig.2C, arrowhead).
                        Contrast echocardiography using saline revealed a patent foramen ovale with right-to-left shunting and bubbles in the left atrium (Fig.2D).
                        The patient underwent an electrophysiologic study with mapping of the accessory pathway, followed by radiofrequency ablation 
                        (interruption of the pathway using the heat generated by electromagnetic waves at the tip of an ablation catheter).
                        His post-ablation ECG showed a prolonged PR interval and an odd “second” QRS complex in leads III, aVF and V2–V4 (Fig.1Bottom), 
                        a consequence of abnormal impulse conduction in the “atrialized” right ventricle.
                        The patient reported no recurrence of palpitations at follow-up 6 months after the ablation.

            Answer:
                        - 28-year-old | True | as it represents the age of the patient (age)
                        - previously healthy | True | as it represents the history of the patient (history)
                        - 6-week | True | as it represents the duration of the symptom palpitations (duration symptom)
                        - palpitations | True | as it one of the symptoms (symptoms)
                        - dyspnea | True | as it is one of the symptoms (symptoms)
                        - electrocardiogram (ECG) | False | as it is not an entity of interest
                        - Ebstein's anomaly | True | as it is the disease being diagnosed (diagnosis) 
                

            Text: {text}
            Answer: 

             """

In [5]:
prompt = ChatPromptTemplate.from_template(template)

model = llm_gpt_35_turbo

In [6]:
chain = (
    {"text": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

In [7]:
text = """The patient was a 34-yr-old man who presented with complaints of fever and a chronic cough.
He was a smoker and had a history of pulmonary tuberculosis that had been treated and cured.
A computed tomographic (CT) scan revealed multiple tiny nodules in both lungs.
A thoracoscopic lung biopsy was taken from the right upper lobe.
The microscopic examination revealed a typical LCH.
The tumor cells had vesicular and grooved nuclei, and they formed small aggregations around the bronchioles (Fig.1).
The tumor cells were strongly positive for S-100 protein, vimentin, CD68 and CD1a.
There were infiltrations of lymphocytes and eosinophils around the tumor cells.
With performing additional radiologic examinations, no other organs were thought to be involved.
He quit smoking, but he received no other specific treatment.
He was well for the following one year.
After this, a follow-up CT scan was performed and it showed a 4 cm-sized mass in the left lower lobe, in addition to the multiple tiny nodules in both lungs (Fig.2).
A needle biopsy specimen revealed the possibility of a sarcoma; therefore, a lobectomy was performed.
Grossly, a 4 cm-sized poorly-circumscribed lobulated gray-white mass was found (Fig.3), and there were a few small satellite nodules around the main mass.
Microscopically, the tumor cells were aggregated in large sheets and they showed an infiltrative growth.
The cytologic features of some of the tumor cells were similar to those seen in a typical LCH.
However, many tumor cells showed overtly malignant cytologic features such as pleomorphic/hyperchromatic nuclei and prominent nucleoli (Fig.4), and multinucleated tumor giant cells were also found.
There were numerous mitotic figures ranging from 30 to 60 per 10 high power fields, and some of them were abnormal.
A few foci of typical LCH remained around the main tumor mass.
Immunohistochemically, the tumor cells were strongly positive for S-100 protein (Fig.5) and vimentin; they were also positive for CD68 (Dako N1577, Clone KPI), and focally positive for CD1a (Fig.6), and they were negative for cytokeratin, epithelial membrane antigen, CD3, CD20 and HMB45.
The ultrastructural analysis failed to demonstrate any Birbeck granules in the cytoplasm of the tumor cells.
Now, at five months after lobectomy, the patient is doing well with no significant change in the radiologic findings.
"""

In [8]:
entities = chain.invoke(text)
print(entities)

- 34-yr-old | True | as it represents the age of the patient (age)
              - smoker | True | as it represents the history of the patient (history)
              - fever | True | as it is one of the symptoms (symptoms)
              - chronic cough | True | as it is one of the symptoms (symptoms)
              - pulmonary tuberculosis | True | as it is the disease being diagnosed (diagnosis)
              - computed tomographic (CT) scan | False | as it is not an entity of interest
              - multiple tiny nodules in both lungs | True | as it represents a finding in the CT scan (symptoms)
              - thoracoscopic lung biopsy | True | as it is the treatment being performed (treatment)
              - LCH (Langerhans cell histiocytosis) | True | as it is the disease being diagnosed (diagnosis)
              - S-100 protein, vimentin, CD68 and CD1a | True | as they are markers used to identify the tumor cells (diagnosis)
              - sarcoma | True | as it is a potential

In [9]:
import re

lines = entities.split('\n')

for line in lines:
    if "| True |" in line:
        parts = re.split(r'\| True \|', line)
        words_before = parts[0].strip()
        words_in_brackets = re.search(r'\((.*?)\)', parts[1]).group(1)
        
        if ',' in words_before:
            words_list = [word.strip() for word in words_before.split(',')]
            for word in words_list:
                if word != words_list[0]:
                    print(f"- {word} ({words_in_brackets})")
                else:
                    print(f"{word} ({words_in_brackets})") 
        else:
            print(f"{words_before} ({words_in_brackets})")


- 34-yr-old (age)
- smoker (history)
- fever (symptoms)
- chronic cough (symptoms)
- pulmonary tuberculosis (diagnosis)
- multiple tiny nodules in both lungs (symptoms)
- thoracoscopic lung biopsy (treatment)
- LCH (Langerhans cell histiocytosis) (diagnosis)
- S-100 protein (diagnosis)
- vimentin (diagnosis)
- CD68 and CD1a (diagnosis)
- sarcoma (diagnosis)
- lobectomy (treatment)
- tumor cells (entity)
- radiologic findings (symptoms)
